# PS2: Selection of an Uncorrelated Tangent Portfolio
Fill me in.

## Problem statement
* Task 1: Compute the log return array and covariance matrix for a collection of `N = 10` stocks that you select from the historical dataset 
* Task 2: Compute the efficient frontier for your collection of `N = 10` stocks
* Task 3: Compute the capital allocation line connecting the risk-free asset and the tangent portfolio
* Task 4: Use your estimated value for the risk aversion parameter `A` to estimate your investment allocation along the capital allocation line

## Setup

In [4]:
include("Include.jl");

    Updating git-repo `https://github.com/varnerlab/VLDecisionsPackage.jl.git`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-5760-PS2-AssetSelectionProblem-Fall-2023/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5760-PS2-AssetSelectionProblem-Fall-2023/Manifest.toml`
  Activating project at `~/Desktop/julia_work/CHEME-5760-PS2-AssetSelectionProblem-Fall-2023`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLDecisionsPackage.jl.git`
  No Changes to `~/Desktop/julia_work/CHEME-5760-PS2-AssetSelectionProblem-Fall-2023/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5760-PS2-AssetSelectionProblem-Fall-2023/Manifest.toml`


### Load historical dataset
We gathered a daily open-high-low-close `dataset` for each firm in the [S&P500](https://en.wikipedia.org/wiki/S%26P_500) for the past five-trading years (a maximum of `1256` data points per firm). However, not all the firms in the `dataset` have the maximum number of trading days, i.e., some firms are missing information for various reasons; perhaps they were acquired, merged, or delisted, etc. We will exclude these firms from the `dataset`.

We load the price `dataset` by calling the `MyPortfolioDataSet()` function:

In [59]:
dataset = MyPortfolioDataSet() |> x-> x["dataset"];

Row,volume,volume_weighted_average_price,open,close,high,low,number_of_transactions
,Float64,Float64,Float64,Float64,Float64,Float64,Int64
1,2.3287e6,71.975,71.77,72.1,72.18,71.34,18833
2,2.53411e6,72.5642,72.75,72.53,72.9128,72.26,18670
3,2.16626e6,73.4262,72.83,73.36,73.92,72.57,15458
4,3.63175e6,74.1492,73.43,74.24,74.42,73.1607,17579
5,2.7221e6,75.1605,74.7,75.11,75.475,74.465,22500
6,2.25761e6,73.7219,74.58,73.91,74.8,73.28,18808
7,1.62933e6,74.4516,74.05,74.59,74.68,73.72,13584
8,1.91578e6,75.3884,74.91,75.39,75.69,74.77,15920
9,2.59929e6,75.7203,76.06,75.54,76.4,75.21,21597


In [9]:
all_firms = keys(dataset) |> collect |> sort;

In [51]:
all_firms

458-element Vector{Int64}:
   1
   2
   3
   4
   6
   7
   8
   9
  10
  11
  12
  13
  14
   ⋮
 492
 493
 494
 495
 497
 498
 500
 501
 502
 503
 504
 505

While it is sometimes convenient to work with the data using the `firm_index`, often we want to specify the [ticker symbol](https://en.wikipedia.org/wiki/Ticker_symbol#:~:text=A%20ticker%20symbol%20or%20stock,on%20a%20particular%20stock%20market.) instead. To facilitate this, let's load a mapping between the `firm_index` and the ticker symbols using the `MyFirmMappingDataSet()` function:

In [48]:
firm_mapping = MyFirmMappingDataSet()

Row,Symbol,Name,Sector
,String7,String,String31
1,MMM,3M,Industrials
2,AOS,A. O. Smith,Industrials
3,ABT,Abbott Laboratories,Health Care
4,ABBV,AbbVie,Health Care
5,ABMD,Abiomed,Health Care
6,ACN,Accenture,Information Technology
7,ATVI,Activision Blizzard,Communication Services
8,ADM,ADM,Consumer Staples
9,ADBE,Adobe,Information Technology


In [49]:
ticker_lookup_dictionary = Dict{String,Int64}();
number_of_map_elements = nrow(firm_mapping);
for i ∈ 1:number_of_map_elements
    ticker = firm_mapping[i,:Symbol];    
    ticker_lookup_dictionary[ticker] = i;
end

505

In [39]:
all_return_matrix = log_return_matrix(dataset, all_firms; Δt = (1/252));
all_covariance_matrix = (1/100)*cov(all_return_matrix);
all_expected_return = mean(all_return_matrix, dims=1) |> vec |> x-> exp.(x) .- 1;

In [58]:
all_tickers_array = Array{String,1}()
for i ∈ eachindex(all_firms)
    firm_index = all_firms[i];
    push!(all_tickers_array, firm_mapping[firm_index, :Symbol])
end
all_tickers_array

458-element Vector{String}:
 "MMM"
 "AOS"
 "ABT"
 "ABBV"
 "ACN"
 "ATVI"
 "ADM"
 "ADBE"
 "AAP"
 "AMD"
 "AES"
 "AFL"
 "A"
 ⋮
 "WRK"
 "WY"
 "WHR"
 "WMB"
 "WYNN"
 "XEL"
 "XYL"
 "YUM"
 "ZBRA"
 "ZBH"
 "ZION"
 "ZTS"

### Set constants
Finally, we set some constant values that are used in the computation of the efficient frontier and the capital allocation line:

In [7]:
number_of_firms = 10;
initial = zeros(number_of_firms);
initial[1] = 1.0;
bounds = zeros(number_of_firms,2);
bounds[:,2] .= 1.0;
risk_free_rate = 0.045
number_of_points = 20;

## Task 1: Compute the log return array and covariance matrix for your selection of stocks

Each firm is assigned an `index` (the `keys` of the `dataset` dictionary). Let's specify a list of `tickers` and look up the corresponding `firm_index` from the `ticker_lookup_dictionary`. We'll hold the indexes of the firms we are interested in in the `firms` array:

In [66]:
my_list_of_tickers = ["ADBE","ZTS","AAPL"];
my_list_of_firms = Array{Int64,1}();
for ticker ∈ my_list_of_tickers
    firm_index = ticker_lookup_dictionary[ticker];    
    push!(my_list_of_firms, firm_index)
end

In [67]:
my_expected_return_array = Array{Float64,1}();
for i ∈ eachindex(my_list_of_firms)
    firm_index = findfirst(x->x==my_list_of_firms[i], all_firms)
    push!(my_expected_return_array, all_expected_return[firm_index])
end

In [68]:
my_expected_return_array

3-element Vector{Float64}:
 0.1318403601715048
 0.1528055826813759
 0.2448236308641849

## Task 2: Compute the efficient frontier for your collection of `N = 10` stocks
Fill me in

## Task 3: Compute the capital allocation line
Fill me in

## Task 4: Compute your operating point
Fill me in